### 필수 라이브러리

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig 
)
from datasets import load_dataset
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig

from huggingface_hub import login

login(
    token="허깅페이스 토근 입력",
    add_to_git_credential=True
)

### 데이터셋 준비

In [3]:
def formatting_prompts_func(examples):
    eos_token = '<|end_of_text|>' 	# Llama 모델의 eos token
    korQuAD_prompt = """		  	# 학습시킬 Prompt의 형태
        ### Question:
        {}

        ### Context:
        {}

        ### Answer:
        {}
    """
	
    #데이터셋에서 원하는 컬럼을 가져와 instruction, input, output으로 매핑
    instructions = examples["question"]
    inputs = examples["context"]
    outputs = [item['text'][0] for item in examples["answers"]]
    texts = []
	
    #데이터 뒤에 eos_token을붙여서 최종 데이터 가공 
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = korQuAD_prompt.format(instruction, input, output) + eos_token
        texts.append(text)

    return {"text": texts}

In [4]:
dataset = load_dataset("KorQuAD/squad_kor_v1", split = "train")
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,        
)

# 데이터 일부만 사용 (10,000개 샘플만 선택)
dataset = dataset.select(range(10000))

print(dataset[:1])

{'id': ['6566495-0-0'], 'title': ['파우스트_서곡'], 'context': ['1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'], 'question': ['바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'], 'answers': [{'text': ['교향곡'], 'answer_start': [54]}], 'text': ['\t\t  \t# 학습시킬 Prompt의 형태\n        ### Question:\n        바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?\n\n        ### Context:\n        1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내

### 양자화

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # 4bit 양자화
    bnb_4bit_compute_dtype=torch.bfloat16,  # 계산 시 사용할 dtype
    bnb_4bit_use_double_quant=True,         # double quantization 사용
    bnb_4bit_quant_type="nf4"               # nf4 또는 fp4 중 선택 가능
)

### Llama 모델 준비

In [6]:
model_path = "meta-llama/Llama-3.2-1B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = '<|end_of_text|>'

model = prepare_model_for_kbit_training(model) # LoRA 적용을 위한 Quantization이 수행된 모델로 준비
model.gradient_checkpointing_enable() # 메모리 절감 효과과

### 파인튜닝 PEFT

In [7]:
# LoRA 파라미터 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM",
)

# LoRA 적용
model = get_peft_model(model, lora_config)

- r : 어댑터 파라미터의 차원 갯수, 기본값은 8, r값이 증가하면 LoRA 학습 과정에서 업데이트해야하는 매개변수가 증가
- lora_alpha: 어댑터의 스케일링 값, 어댑터에서 나온 output에 곱해지는 값으로 기본값은 8
- lora_dropout: LoRA 레이어의 드롭아웃 확률을 의미하며, 기본값은 0, 드롭아웃은 과적합을 줄이기 위해 무작위로 뉴런을 선택해 생략하는 방식
- target_modules: LoRA 학습에서 모델 아키텍쳐 중 타게팅할 모듈 리스트, 많은 모듈을 타게팅하면 훈련 시간이 증가, 주로 transformer의 어텐션 블록만 타게팅함
- bias: LoRA 바이아스 유형, 기본값은 None

### 학습 파라미터 설정

In [8]:
training_params = SFTConfig( #
    output_dir="/results",
    num_train_epochs=3,	# 기본값은 3
    per_device_train_batch_size=1,	# 기본값은 8
    gradient_accumulation_steps=1,	# 기본값 1
    optim="paged_adamw_32bit",
    save_steps=10000,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    dataset_text_field="text", #
    max_seq_length=512, #
    packing=False, #
)

### 모델 학습

In [9]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    tokenizer=tokenizer,
    args=training_params,
)

C:\Users\user\AppData\Local\Temp\ipykernel_13416\2361144306.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/10000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [10]:
trainer.train()

# 어댑터 저장
ADAPTER_MODEL = "lora_adapter_test1"
trainer.model.save_pretrained(ADAPTER_MODEL)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: zuyeonn (zuyeonn-sungshin-women-s-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
25,3.504500
50,3.239200
75,3.052500
100,3.129000
125,3.157300
150,3.122200
175,3.174000
200,3.098800
225,3.275400
250,3.233700


In [ ]:
# GPU 1번으로 할당
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"